### This notebook analyses the IWP retreived using QRNN. Uses test data (simulations)

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import ipywidgets as w
import matplotlib.pyplot as plt
import numpy as np
import netCDF4
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from torch.utils.data import DataLoader, random_split
from iwc2tb.GMI.gmiData import gmiData
from iwc2tb.GMI.GOES import GOES
import os
from mpl_toolkits.basemap import Basemap, cm
import numpy.ma as ma
from iwc2tb.common.hist2d import hist2d
from iwc2tb.common.plot_locations_map import plot_locations_map
import matplotlib.colors as colors
from matplotlib import cm
#from typhon.retrieval.qrnn import set_backend, QRNN
#set_backend("pytorch")

### Read in input data

In [2]:
batchSize          = 256
inputs             = np.array( ["ta", "t2m",  "wvp", "z0", "lat", "stype"])
ninputs            = len(inputs) + 3
outputs            = "iwp"
xlog               = True
latlims            = [0, 45]
latlims            = [0, 65]
#quantiles          = np.arange(0.05, 1, 0.05)
quantiles         = np.linspace(0.01, 0.99, 50)
imedian            = np.argwhere((quantiles >= 0.49) & (quantiles < 0.51))[0][0]



filename = '~/Dendrite/Projects/IWP/GMI/training_data/try_training/qrnn_gmi_nn_lpa_v1.nc'

#filename = '~/Dendrite/Projects/IWP/GMI/training_data/try_training/qrnn_gmi_all_hotencode.nc'



training_data      = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_train_jan_esa.nc"), 
                             inputs, 
                             outputs,
                             batch_size = batchSize,
                             latlims = latlims,)
                             #log_iwp = xlog)

norm = training_data.norm

In [3]:

validation_data    = gmiData(os.path.expanduser("~/Dendrite/Projects/IWP/GMI/training_data/TB_GMI_train_jan_pr_aro_new.nc"), 
                             inputs,
                             outputs,
                             batch_size = batchSize,
                             latlims = latlims,
                             normalise = norm,)
                             #log_iwp = xlog)


In [4]:
def get_pos_mean(validation_data, qrnn, quantiles, log = False):
    y          = []
    y_pos_mean = []
    y_pre      = []
    x_in       = []

    nbatch = len(validation_data)
    for i in range(nbatch):

        xx, yy = validation_data[i]
        x = xx.detach().numpy() 
        y_pre.append(qrnn.predict(xx)) 
        #y_pos_mean.append((posterior_mean(x, qrnn, quantiles, log)))
        y_pos_mean.append(qrnn.posterior_mean(xx))
        y.append(yy.detach().numpy())
        x_in.append(x)

    y_pre = np.concatenate(y_pre, axis = 0)
    y = np.concatenate(y, axis= 0)
    y_pos_mean = np.concatenate(y_pos_mean, axis = 0) 
    
    #if log == True:
    #    y      = np.exp(y)
    #    y_pre  = np.exp(y_pre)

        
    return y, y_pre, y_pos_mean

def cdf(x, qrnn, quantiles, log = False):
    if len(x.shape) > 1:
        s = x.shape[:-1] + (quantiles.size + 2,)
    else:
        s = (1, quantiles.size + 2)

    y_pred = np.zeros(s)
    pre    = qrnn.predict(x)
    
    if log == True:
        pre = np.exp(pre)
        
    y_pred[:, 1:-1] = pre

    y_pred[:, 0] = 2.0 * y_pred[:, 1] - y_pred[:, 2]
    y_pred[:, -1] = 2.0 * y_pred[:, -2] - y_pred[:, -3]

    qs = np.zeros(quantiles.size + 2)
    qs[1:-1] = quantiles
    qs[0] = 0.0
    qs[-1] = 1.0

    return y_pred, qs


def posterior_mean(x, qrnn, quantiles, log = False):
    y_pred, qs = cdf(x, qrnn, quantiles, log)
    mus = y_pred[:, -1] - np.trapz(qs, x=y_pred)
    return mus


In [5]:
norm.stats

{0: (268.5954546635614, 20.275572694774606),
 1: (265.01122187242123, 23.294676778521556),
 2: (265.47096955555645, 13.746809194371663),
 3: (257.07374133613956, 9.433577286174387),
 4: (286.3308850128538, 14.495210294872672),
 5: (22.579110466154248, 16.379016850531343),
 6: (163.62902746536665, 450.5018663843802),
 7: (0.43568426363561935, 37.64977619589557)}

In [6]:
validation_data.pr

array([[1.14027772],
       [1.18403939],
       [1.0509067 ],
       ...,
       [1.23001965],
       [1.10990205],
       [1.09860334]])

In [7]:
from quantnn.qrnn import QRNN
import pickle

### load training

In [8]:

for i in ["0", "1", "2", "3"]:
    
        
    filename          = '~/Dendrite/Projects/IWP/GMI/training_data/try_training/' + "qrnn_gmi_nn_esa_v" + i + ".nc"
    print(filename)
    qrnn              =  QRNN.load(os.path.expanduser(filename))
    y1, y_pre1, y_pos_mean1 = get_pos_mean(validation_data, qrnn, quantiles, log = xlog)


    outfile = os.path.basename(filename)[:-3] + ".pickle"
    with open(outfile, "wb") as f:

        pickle.dump(y1, f)
        pickle.dump(y_pre1, f)
        pickle.dump(y_pos_mean1, f)
        pickle.dump(validation_data.pr, f)

        f.close()


~/Dendrite/Projects/IWP/GMI/training_data/try_training/qrnn_gmi_nn_esa_v0.nc
~/Dendrite/Projects/IWP/GMI/training_data/try_training/qrnn_gmi_nn_esa_v1.nc
~/Dendrite/Projects/IWP/GMI/training_data/try_training/qrnn_gmi_nn_esa_v2.nc
~/Dendrite/Projects/IWP/GMI/training_data/try_training/qrnn_gmi_nn_esa_v3.nc


In [9]:
y_pre1

array([[8.0839931e-07, 1.1303722e-06, 1.2429388e-06, ..., 4.8364006e-04,
        9.9657825e-04, 5.5218842e-03],
       [5.2942872e-01, 7.6380765e-01, 7.8231150e-01, ..., 1.3973188e+00,
        1.5708256e+00, 2.2805333e+00],
       [5.5668652e-03, 2.9820139e-02, 6.1319686e-02, ..., 1.1254177e+00,
        1.6050744e+00, 2.0607188e+00],
       ...,
       [2.2674303e-06, 8.0569898e-06, 2.0512980e-05, ..., 5.7063520e-02,
        7.8212447e-02, 1.7998292e-01],
       [1.9609701e-07, 3.0324186e-06, 1.5390320e-05, ..., 6.5890677e-02,
        9.3570262e-02, 2.1237646e-01],
       [7.9098197e-07, 1.1897635e-06, 1.6263364e-06, ..., 8.3316587e-02,
        1.1819479e-01, 1.4632618e-01]], dtype=float32)

In [10]:
a = y_pre1[:, imedian]

plt.hist(a[y1< 0.02], bins = np.arange(0, 0.5, 0.01), histtype = "step")
plt.hist(y1[y1< 0.02], bins = np.arange(0, 0.5, 0.01), histtype = "step")


IndexError: too many indices for array

### PDFs of predictions

In [ ]:
plt.rcParams.update({'font.size': 20})
#bins1 = np.arange(-1, 20, 0.1)

bins1 = np.array([0.0,.0001,.00025,.0005, 0.001,.0025,.005,
                 0.0075, 0.01, 0.025, 0.05, 0.075, .1, .25,
                 .5, .75, 1, 2, 3, 4, 5, 6, 8, 10, 12,13, 14,15 ,16, 20, 24])

bin_center = (bins1[1:] + bins1[:-1])/2
fig, axes = plt.subplots(1, 2, figsize = [12, 6])
for ax, a, title in zip(axes.ravel(), 
                        [y_pre1[:, imedian], y_pos_mean1], 
                        ['y_median', 'y_mean']):
    

    
    hist_y, _     = np.histogram(y1, bins1, density = True)
    hist_y_pre, _ = np.histogram(a, bins1, density = True)
    
    ax.plot(bins1[1:], hist_y, label = "observed")
    ax.plot(bins1[1:], hist_y_pre, label = "predicted")
    
    
    #ax.hist(a, bins1, density = True , histtype = "step", label = "QRNN")
    #ax.hist(np.exp(y1), bins1, density = True, histtype = "step", label = "GMI")
    
    ax.set_yscale('log')
    ax.set_xscale('log')
    
    ax.set_title(title)
    ax.legend()
    ax.grid()
    ax.set_ylabel("PDF")
    ax.set_xlabel("IWP[kg/m2]")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [6, 6])
rndinds = np.random.randint(1, 8000, 4000)
for i in rndinds:
    ax.plot(quantiles, y_pre1[i, :], 'b', alpha = 0.2)
ax.set_xlabel("quantiles")
ax.set_ylabel("IWP")
#fig.savefig("quantiles.png")

In [ ]:
tb = validation_data.x[:, :4]
pd  = (tb[:, 0] - tb[:, 1])
pdmask = (pd > 10)
tbmask  = (tb[:, 0] < 230  )
mask = np.logical_and(pdmask, tbmask) 


In [ ]:
plt.scatter(tb[mask, -1], pd[mask])

In [ ]:
plt.scatter(y1[mask], y_pos_mean1[mask], alpha = 0.2)
x = np.arange(0.1, 15, 1)
y = x
plt.plot(x, y)
plt.yscale("log")
plt.xscale("log")

In [ ]:
def sharpness(y):
    s = np.mean((y[mask, -13 ] - y[mask, 12])/y[mask, imedian])
    return s

In [ ]:
np.mean(y_pre1[mask, imedian] - y1[mask])

In [ ]:
quantiles[12]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [6, 6])

for i in range(y1[mask].shape[0]):
    ax.plot(quantiles, y_pre1[mask, :][i], 'b', alpha = 0.2)
ax.set_xlabel("quantiles")
ax.set_ylabel("IWP")
#fig.savefig("quantiles.png")

In [ ]:
plt.scatter(y_pre1[:, imedian], y1)

### scatter plot of predictions vs simulated

In [ ]:
import scipy
from matplotlib import ticker, cm
xyrange = [[0, 15], [0, 15]] # data range
bins = [45, 45] # number of bins

fig, axes = plt.subplots(1, 2, figsize = [12, 6])
for ax, a, a0, title in zip(axes.ravel(), 
                     [y_pre1[:, imedian], y_pos_mean1], 
                     [y1, y1],
                    ['y_median', 'y_mean']):

   
    hh, locx, locy = np.histogram2d(a0, a, 
                                    range=xyrange, 
                                    bins=bins, density = True)

    posx = np.digitize(a0, locx)
    posy = np.digitize(a, locy)
    cs = ax.contourf(np.flipud(hh.T),
                    extent=np.array(xyrange).flatten(), 
                locator= ticker.LogLocator(), origin='upper')
    #cbar = fig.colorbar(cs)
    ax.set_ylim([0, 12])
    ax.set_xlim([0, 12])
    ax.set_title(title)
    xy = np.arange(0, 13, 1)
    yy = xy
    ax.plot(xy, yy)
    ax.set_ylabel("IWP retrieved [kg/m2]")
    ax.set_xlabel("IWP observed [kg/m2]")


### analysis of IWP fields

In [ ]:
def plot_iwp(lat, lon, iwp0, iwp, iwp1, tb, mask):
    
    fig, ax = plt.subplots(1, 3, figsize = [15, 8])
    fig.tight_layout(pad=3.0)
    ax = ax.ravel()
    diff = 100 * (np.exp(np.abs(np.log(iwp/iwp0))) - 1)
    
    bbox = [np.min(lon),np.min(lat),np.max(lon),np.max(lat)] # set bounds for plotting
    n_add = 0
    m = Basemap(llcrnrlon=bbox[0]-n_add,llcrnrlat=bbox[1]-n_add,
                urcrnrlon=bbox[2]+n_add,urcrnrlat=bbox[3]+n_add,resolution='l',
                projection='cyl')
    x, y = m(lon, lat)    
    for var, axes, t in zip([iwp0, iwp, iwp1], ax, ["Simulated", " y_median", "y_mean"]):
        
        cs = axes.scatter(lon[mask],lat[mask], c = var[mask]* 1000,
                          norm=colors.LogNorm(), vmin = 1, vmax = 10000,)
                        # cmap = cm.Paired)
            
        #cs = m.scatter(lon, lat, var[mask]* 1000, ax = axes)    
        axes.set_title(t)
        ax[0].set_ylabel("Latitude [deg]")
        #axes.set_xlabel("Longitude [deg]")
    cbar = fig.colorbar(cs, ax=[ax[0], ax[1], ax[2]])
    cbar.ax.set_ylabel("IWP [g/m2]") 

def get_mask(lat, lon, latlims, lonlims):
    
    im  = (lat >= latlims[0]) & (lat <= latlims[1])
    im1 = (lon >=  lonlims[0]) & (lon < lonlims[1])
    mask  = np.logical_and(im, im1)
    
    return mask    

def get_coords(validation_data):
    lat = validation_data.lat
    lon = validation_data.lon%360
    tb  = validation_data.x[:, :5]
    stype = validation_data.x[:, -2]

    
    return lat, lon, tb, stype

### spatial analysis IWP

In [ ]:
lat1,  lon1, tb1, stype1    = get_coords(validation_data)
lonlims = [20, 30]
latlims = [0, 30]
mask1 = get_mask(lat1, lon1, latlims, lonlims)
plot_iwp(lat1, lon1, y1, y_pre1[:, imedian], y_pos_mean1, tb1, mask1)  

lonlims = [0, 360]
latlims = [-60, 60]
mask1 = get_mask(lat1, lon1, latlims, lonlims)
plot_iwp(lat1, lon1, y1, y_pre1[:, imedian], y_pos_mean1, tb1, mask1)  

### compare histograms of y0 - y_pre

In [ ]:
bins = np.arange(-2.5, 2.5, 0.05)
fig, ax = plt.subplots(1, 1, figsize = [8, 6])
hist_median, _ = np.histogram(y1 - y_pre1[:, imedian], bins, density = True)
hist_mean, _   = np.histogram(y1 - y_pos_mean1 , bins, density = True)
xbins = (bins[1:] + bins[:-1])/2.
ax.plot(xbins, hist_median, label = "y_median")
ax.plot(xbins, hist_mean, label = "y_mean")
ax.set_yscale('log')
ax.legend()
ax.set_xlabel("y0 - y_pre [kg m$^{-2}$]")
ax.set_ylabel("PDF")
plt.grid(True)
plt.show()
fig.savefig("PDF_IWP.png", bbox_inches = "tight")

###  associated  uncertainty

In [ ]:

fig, ax = plt.subplots(1, 1, figsize = [6, 6])
rndinds = np.random.randint(1, 30000, 1500)
for i in rndinds:
    ax.plot(quantiles, y_pre1[i, :], 'b', alpha = 0.2)
ax.set_xlabel("quantiles")
ax.set_ylabel("IWP")
fig.savefig("quantiles.png")

In [ ]:
y1.shape

In [ ]:
fig, axes = plt.subplots(2, 2, figsize = [8, 8])
fig.tight_layout(pad=3.0)

for ax, ix, title in zip(axes.ravel(), [0, 1, 2, 3], ["water", "land", "snow", "sea-ice"]):
    mask = stype1 == ix
    ax.scatter(y1[mask], y_pos_mean1[mask])
    ax.set_title(title)
    ax.set_ylabel("Retrieved")
    ax.set_xlabel("Simulated") 

#### 2d histogram y0 and y_pre on log scales (see ISMAR article for reference)

In [ ]:
im = stype1 == 0
im1 = stype1 == 2
xyrange = [[-5, 2], [-5, 20]]
#xyrange = [[1e-5, 1e2], [1e-5, 1e20]]
bins = [50, 120]
xdat1 =  (np.log10(y_pos_mean1))
ydat1 = (np.log10((y_pos_mean1/y1)))

xdat = (np.log10(y_pre1[:, imedian]))
ydat = (np.log10((y_pre1[:, imedian]/y1)))


fig, axes = plt.subplots(1, 2, figsize = [12, 6])
fig.tight_layout(pad=3.0)    
for ax, xdt, ydt, title in zip(axes.ravel(), [xdat, xdat1], [ydat, ydat1], ["median", "mean"]):
        
        hh, xyrange, xdt1, ydt1 = hist2d(xdt, ydt, 
                                           bins = bins, xyrange = xyrange)
        cs = ax.contourf(np.flipud(hh.T), 
                         levels=[0.5e-3, 1e-3,  0.5e-2, 1e-2,  0.5e-1, 1e-1, 0.5,  1e0], 
                         cmap= 'Blues',
                extent=np.array(xyrange).flatten(), 
            locator= ticker.LogLocator(), origin='upper')
        
        
        hh, xyrange, xdt1, ydt1 = hist2d(xdt[im1], ydt[im1], 
                                           bins = bins, xyrange = xyrange)
        cs1 = ax.contour(np.flipud(hh.T), 
                         levels=[0.5e-3, 1e-3,  0.5e-2, 1e-2,  0.5e-1, 1e-1, 0.5,  1e0], 
                         cmap= 'Reds',
                extent=np.array(xyrange).flatten(), 
            locator= ticker.LogLocator(), origin='upper')
        
        
        

        hh, locx, locy = np.histogram2d(xdt, ydt, range = xyrange,
                                     bins = bins, density = True)
        dy = 0.5 * (locy[1:] + locy[:-1])
        dx = 0.5 * (locx[1:] + locx[:-1])
        off = []

        for i in range(50):
            a = np.sum(dy * hh[i, :])
            b = np.sum(hh[i, :])
            off.append(a/b)
        off = np.stack(off)    
        ax.plot(dx, off, 'k')
        ax.set_ylim([-5, 10])
        ax.grid('on')

        ax.set_ylabel(r"log10(y_pre / y0)]")
        ax.set_xlabel(r"log10(y_pre)")
        ax.set_title(title)
cbar = fig.colorbar(cs, ax = axes)  

fig.savefig("hist2d_IWP_hlats.png", bbox_inches = "tight")

### mean fractional error

In [ ]:
def mfe( y0, y):
    return np.median(10**(np.abs(np.log10(y/y0))) - 1) * 100

def calculate_mfe(y0, y):
    nbins = 60
    logbins = np.log10(np.logspace(np.log10(1e-4), np.log10(1e2), nbins))
    ibins = np.digitize(np.log10(y0), logbins)
    err = []
    for ix in range(nbins):

        ix = np.where(ibins == ix)[0]

        err.append(mfe(y0[ix], y[ix]))
    return err, logbins

In [ ]:
y_pre1[y_pre1<0] = 0
err, logbins  = calculate_mfe(y1, y_pos_mean1)
err1, _ = calculate_mfe(y1, y_pre1[:, imedian])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [6, 6])
fig.tight_layout(pad=1.0)   


ax.plot(10 ** (logbins[1:]), err[1:], '-o', label = "mean")
ax.plot(10 **(logbins[1:]), err1[1:], '-o', label = "median")
ax.set_ylim([0, 1000])
ax.set_xscale("log")
#ax.set_title()
ax.set_xlabel(r" IWP [kg m$^{-2}$] ")
ax.set_ylabel("Median fractional error [%]")
ax.legend()   
ax.grid('on')
fig.savefig("median_fractional_error.png", bbox_inches = "tight")

### RMSE and BIAS over stypes

In [ ]:
def rmse(y, y0):
    return np.sqrt(np.mean((y - y0)**2))

def mae(y, y0):
    return np.mean(np.abs(y - y0))

def bias(y, y0):
    return np.mean(y - y0)

In [ ]:
stype1

In [ ]:
rms = []
rms1 = []

bs = []
bs1 = []
stypes = ["water", "land", "snow", "seaice", "coastlines"]
for ix in [0, 1, 2, 3, 4]:
    im = stype1 == ix
    im1 = y1 > 0.01
    im = np.logical_and(im, im1)
    rms.append(rmse(y_pre1[im, imedian], y1[im]))
    rms1.append(rmse(y_pos_mean1[im], y1[im]))
    
    bs.append(bias(y_pre1[im, imedian], y1[im]))
    bs1.append(bias(y_pos_mean1[im], y1[im]))

fig, axes = plt.subplots(2, 1, figsize = [12, 8])
x = np.arange(len(stypes))  # the label locations
width = .20  # the width of the bars
for ax, r, b in zip(axes.ravel(), [rms, bs], [rms1, bs1]):
    print(r)
    ax.bar(x - width/2, r, width = width, label = "median", color = "tab:blue")
    ax.bar(x + width/2, b, width = width, label = "mean", color = "tab:orange")
    ax.set_xticks(x)
    ax.set_xticklabels(stypes)
    ax.legend()
    ax.grid("on")
axes[0].set_ylabel(r"RMSE [kg m$^{-2}$]")
axes[1].set_ylabel(r"Bias [kg m$^{-2}$]")

fig.suptitle("statistics for IWP >= 0.01 kg/m2")
fig.savefig("RMSE_bias_hlats.png", bbox_inches = "tight")


In [ ]:
rms = []
rms1 = []

bs = []
bs1 = []
stypes = ["water", "land", "snow", "seaice", "coastlines"]
for ix in [0, 1, 2, 3, 4]:
    im = stype1 == ix
    #im1 = y1 > 0.01
    #im = np.logical_and(im, im1)
    rms.append(rmse(y_pre1[im, imedian], y1[im]))
    rms1.append(rmse(y_pos_mean1[im], y1[im]))
    
    bs.append(bias(y_pre1[im, imedian], y1[im]))
    bs1.append(bias(y_pos_mean1[im], y1[im]))

fig, axes = plt.subplots(2, 1, figsize = [12, 8])
x = np.arange(len(stypes))  # the label locations
width = .20  # the width of the bars
for ax, r, b in zip(axes.ravel(), [rms, bs], [rms1, bs1]):
    print(r)
    ax.bar(x - width/2, r, width = width, label = "median", color = "tab:blue")
    ax.bar(x + width/2, b, width = width, label = "mean", color = "tab:orange")
    ax.set_xticks(x)
    ax.set_xticklabels(stypes)
    ax.legend()
    ax.grid("on")
axes[0].set_ylabel(r"RMSE [kg m$^{-2}$]")
axes[1].set_ylabel(r"Bias [kg m$^{-2}$]")

fig.suptitle("statistics for all IWP")
fig.savefig("RMSE_bias_hlats_all.png", bbox_inches = "tight")

#### Error over IWP bins

In [ ]:
nbins = 50
logbins = np.logspace(np.log10(1e-3), np.log10(0.5e2), nbins)
ibins = np.digitize(y1, logbins)
counts = np.bincount(ibins)
ntrue = []
for j in range(nbins):
    itrue = ([i for i in range(y1.shape[0]) if y1[i] >= y_pre1[i, 0] if y1[i] <= y_pre1[i, -1] if ibins[i] == j])
    ntrue.append(len(itrue))
ntrue = np.array(ntrue)

fig, ax = plt.subplots(1, 1, figsize = [6, 6])
ax.plot(logbins[:len(counts)], 100 * ntrue[:len(counts)]/counts, '-o')
ax.set_xscale("log")
ax.set_xlabel("IWP [kg/m2]")
ax.set_ylabel("% true occurences in 98% CI")
ax.grid('on')
ax.set_ylim([0, 100])
fig.savefig("Uncertainty_IWP.png", bbox_inches = "tight") 
    

In [ ]:
nbins = 50
logbins = np.logspace(np.log10(1e-8), np.log10(0.5e2), nbins)
ibins = np.digitize(y1, logbins)
imatrix = np.zeros([nbins, nbins])
ibins_p = np.digitize(y_pos_mean1, logbins)


for i in range(ibins.max()):
    in2 = ibins_p[ibins == i].max()
    ib = np.bincount(ibins_p[ibins == i])
    imatrix[ :in2 + 1, i] = 100 * ib/np.sum(ibins == 1)
    
fig, ax = plt.subplots(1, 1, figsize = [7, 6])
imatrix[imatrix == 0] = np.nan
cs = ax.pcolormesh(logbins, logbins, imatrix, cmap='Blues', vmin = 0, vmax = 50)
ax.set_yscale('log')
ax.set_xscale('log')
x = logbins
y = x
ax.plot(x, y, alpha = 0.1)
fig.colorbar(cs)
ax.set_xlabel("IWP simulated [kg/m2]")
ax.set_ylabel("IWP retrieved [kg/m2]")
ax.set_xlim(1e-8, 50)
ax.set_ylim(1e-8, 50)
fig.savefig("histogram2d_IWP.png", bbox_inches = "tight")


### check false hits/misses

In [ ]:
iwplim = 0.01 #[kg/m2]

falsemiss = y1[y_pos_mean1 <= iwplim] >= iwplim

falsehits = y_pos_mean1[y1 <= iwplim] >= iwplim

print("falsemiss among total data : ", np.sum(falsemiss)/y1.shape[0], "\n",
      "falsehits among total data : ", np.sum(falsehits)/y1.shape[0])


print("falsemiss among IWP < 0.01 kg/m2 : ", np.sum(falsemiss)/np.sum(y1[y_pos_mean1 <= iwplim]), "\n",
      "falsehits among IWP < 0.01 kg/m2 : ", np.sum(falsehits)/np.sum(y_pos_mean1[y1 <= iwplim]))



In [ ]:
y_pos_mean1.min()


In [ ]:
logbins

In [ ]:
outfile1 = "qrnn_gmi_all_z0_log10.pickle"
outfile2 = "qrnn_gmi_all_z0.pickle"
outfile3 = "qrnn_gmi_all_z0_linear.pickle"

In [ ]:
def read_pickle(file):
    with open(file, "rb") as f:
        y      = pickle.load(f)
        y_pre  = pickle.load(f)
        y_mean = pickle.load(f)
        f.close()
        
    return y, y_pre, y_mean

y1, y_pre1, y_mean1  = read_pickle(outfile1)
y2, y_pre2, y_mean2  = read_pickle(outfile2)
y3, y_pre3, y_mean3  = read_pickle(outfile3)
    

In [ ]:
def stats(y1, y_pre1, y_pos_mean1):
    rms = []
    rms1 = []

    bs = []
    bs1 = []
    stypes = ["water", "land", "snow", "seaice", "coastlines"]
    for ix in [0, 1, 2, 3, 4]:
        im = stype1 == ix
        #im1 = y1 > 0.01
        #im = np.logical_and(im, im1)
        rms.append(rmse(y_pre1[im, imedian], y1[im]))
        rms1.append(rmse(y_pos_mean1[im], y1[im]))

        bs.append(bias(y_pre1[im, imedian], y1[im]))
        bs1.append(bias(y_pos_mean1[im], y1[im]))
        
    return rms, bs   

In [ ]:
stypes = ["water", "land", "snow", "seaice", "coastlines"]
rms1, bs1 = stats(y1, y_pre1, y_mean1)
rms2, bs2 = stats(y2, y_pre2, y_mean2)
rms3, bs3 = stats(y3, y_pre3, y_mean3)

In [ ]:
x = np.arange(0, len(stypes), 1)
width = 0.15
fig, ax = plt.subplots(1, 1, figsize = [10, 6])
ax.bar(x - 0.2, rms1, width,  label = "log10")
ax.bar(x , rms2, width, label = "log")
ax.bar(x + 0.2, rms3, width, label = "linear")
ax.set_xticks(x)
ax.set_xticklabels(stypes)
ax.legend()


In [ ]:
x = np.arange(0, len(stypes), 1)
width = 0.15
fig, ax = plt.subplots(1, 1, figsize = [10, 6])
ax.bar(x - 0.2, bs1, width,  label = "log10")
ax.bar(x , bs2, width, label = "log")
ax.bar(x + 0.2, bs3, width, label = "linear")
ax.set_xticks(x)
ax.set_xticklabels(stypes)
ax.legend()

In [ ]:
err1, logbins  = calculate_mfe(y1, y_mean1)
err2, _        = calculate_mfe(y2, y_mean2)
err3, _        = calculate_mfe(y3, y_mean3)

fig, ax = plt.subplots(1, 1, figsize = [6, 6])
fig.tight_layout(pad=1.0)   


ax.plot(10 ** (logbins[1:]), err1[1:], '-o', label = "log10")
ax.plot(10 **(logbins[1:]), err2[1:], '-o', label = "log")
ax.plot(10 **(logbins[1:]), err3[1:], '-o', label = "linear")
ax.set_ylim([0, 1000])
ax.set_xscale("log")
#ax.set_title()
ax.set_xlabel(r" IWP [kg m$^{-2}$] ")
ax.set_ylabel("Median fractional error [%]")
ax.legend()   
ax.grid('on')
fig.savefig("median_fractional_error.png", bbox_inches = "tight")

In [ ]:
def true_events(y1, y_pre1):
    nbins = 50
    logbins = np.logspace(np.log10(1e-3), np.log10(0.5e2), nbins)
    ibins = np.digitize(y1, logbins)
    counts = np.bincount(ibins)
    ntrue = []
    for j in range(nbins):
        itrue = ([i for i in range(y1.shape[0]) if y1[i] >= y_pre1[i, 0] if y1[i] <= y_pre1[i, -1] if ibins[i] == j])
        ntrue.append(len(itrue))
    ntrue = np.array(ntrue)
    
    return ntrue, counts, logbins


ntrue1, counts1, logbins = true_events(y1, y_pre1)
ntrue2, counts2, _ = true_events(y2, y_pre2)
ntrue3, counts3, _ = true_events(y3, y_pre3)


fig, ax = plt.subplots(1, 1, figsize = [6, 6])
ax.plot(logbins[:len(counts1)], 100 * ntrue1[:len(counts1)]/counts1, '-o', label = "log10")
ax.plot(logbins[:len(counts2)], 100 * ntrue2[:len(counts2)]/counts2, '-o', label = "log")
ax.plot(logbins[:len(counts3)], 100 * ntrue3[:len(counts3)]/counts3, '-o', label = "linear")

ax.set_xscale("log")
ax.set_xlabel("IWP [kg/m2]")
ax.set_ylabel("% true occurences in 98% CI")
ax.grid('on')
ax.set_ylim([60, 100])
ax.legend()
fig.savefig("Uncertainty_IWP.png", bbox_inches = "tight") 